In [1]:
from bs4 import BeautifulSoup
import urllib3
from urllib.request import urlopen
import smtplib
from flask import Flask, request, Response
import re
import time 
from textblob import TextBlob

In [2]:
def make_soup(url):
    html = urlopen(url)
    return BeautifulSoup(html, 'html.parser')

In [31]:
# exportar los datos de los procesadores a un csv

import csv
import requests
from bs4 import BeautifulSoup

def make_soup(url):
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup

url = "https://www.pc-kombo.com/us/components/cpus"
soup = make_soup(url)

procesadores = []
for i in soup.find("ol", {'id': 'hardware'}).find_all('li'):
    if(i != 'None' and i != -1):
        nombre = i.find("h5", {'class': 'name'})
        subtitle = i.find("div", {'class': 'subtitle'})
        turbo = ''
        clock = ''
        threads = ''
        cores = ''
        price = ''
        socket = None  # Inicializar la variable socket
        # buscar el valor del socket
        socket_html = subtitle.find("span", {'class': 'socket'})
        if socket_html is not None:
            socket = socket_html.text.strip()
        for atributo in subtitle.find_all('span'):
            if atributo.text.startswith('Turbo'):
                turbo = atributo.text.split(' ')[1]
            elif atributo.text.startswith('Clock'):
                clock = atributo.text.split(' ')[1]
            elif atributo.text.endswith('Threads'):
                threads = atributo.text.split(' ')[0]
            elif atributo.text.endswith('Cores'):
                cores = atributo.text.split(' ')[0]
        # buscar el precio en el HTML
        precio_html = i.find("span", {'class': 'price'})
        if precio_html is not None:
            # obtener el valor del precio y asignarlo a la variable price
            price = precio_html.text.strip()[4:]
    
        if price !='':
            url_cpu = (i.find('a', href=True)['href']).split(" ")[0]
            response = requests.get(url_cpu)

            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Obtener el performance level
            perf_level = soup.find('p', {'itemprop': 'aggregateRating'})
            if perf_level is not None:
                perf_level = perf_level.text
                perf_level_pattern = re.compile(r'(\d+\.\d+/\d+)')
                perf_level_match = perf_level_pattern.search(perf_level)

                if perf_level_match is not None:
                    perf_level = perf_level_match.group(1)
                    perf_level = perf_level[:-3]
                    
            # Obtener el productor
            producer = soup.find("dd",{'itemprop':'brand'} )
            if producer is not None:
                    producer = producer.text
            
            procesador = {
            'nombre': nombre.get_text(strip=True),
            'socket': socket,
            'turbo': turbo,
            'clock': clock,
            'threads': threads,
            'cores': cores,
            'perf_level' : perf_level,
            'producer': producer,
            'precio': price
            }
            
            procesadores.append(procesador)

# Escribir los datos en un archivo CSV
with open('procesadores.csv', mode='w', newline='', encoding='utf-8') as outfile:
    writer = csv.writer(outfile)
    # Escribir la cabecera del archivo CSV
    writer.writerow(['Nombre', 'Socket', 'Turbo', 'Clock', 'Threads', 'Cores','Performance Level /10','Producer', 'Precio'])
    # Escribir cada fila en el archivo CSV
    for procesador in procesadores:
        writer.writerow([procesador['nombre'], procesador['socket'], procesador['turbo'], 
                         procesador['clock'], procesador['threads'], procesador['cores'], 
                         procesador['perf_level'],procesador['producer'], procesador['precio']])


In [32]:
# exporta los datos necesarios de las gpus
import csv
import requests
import re
from bs4 import BeautifulSoup

def make_soup(url):
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup

url = "https://www.pc-kombo.com/us/components/gpus"
soup = make_soup(url)

gpus = []
for i in soup.find("ol", {'id': 'hardware'}).find_all('li'):
    if(i != 'None' and i != -1):
        nombre = i.find("h5", {'class': 'name'})
        subtitle = i.find("div", {'class': 'subtitle'}).find_all('span')
        serie = ''
        vram = ''
        potencia = ''
        serie =i.find("span", {'class': 'series'}).text
        vram = i.find("span", {'class': 'vram'}).text[:-2]
        for atributo in subtitle:
            if atributo.text.endswith('W'):
                potencia = atributo.text[:-1]
                
        # buscar el precio en el HTML
        precio_html = i.find("span", {'class': 'price'})
        if precio_html is not None:
            # obtener el valor del precio y asignarlo a la variable price
            price = precio_html.text.strip()[4:]  
            
        if price != '':
            url_ram = (i.find('a', href=True)['href']).split(" ")[0]
            response = requests.get(url_ram)

            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Obtener el productor
            producer = soup.find("dd",{'itemprop':'brand'} )
            if producer is not None:
                producer = producer.text
            

            #obtener el length
            length = soup.find('dt', text='Length')
            if length is not None:
                length = length.find_next_sibling('dd').text[:-2]

            #obtener los slots
            slots = soup.find('dt', text='Slots')
            if slots is not None:
                slots = slots.find_next_sibling('dd').text 
            
            #obtener los 8 pin
            eight_pin = soup.find('dt', text='8-pin connectors')
            if eight_pin is not None:
                eight_pin = eight_pin.find_next_sibling('dd').text
            else:
                eight_pin = 0

            #obtener los 6 pin
            six_pin = soup.find('dt', text='6-pin connectors')
            if six_pin is not None:
                six_pin = six_pin.find_next_sibling('dd').text
            
            #obtener el boost clock
            boost_clock = soup.find('dt', text='Boost Clock')
            if boost_clock is not None:
                boost_clock = boost_clock.find_next_sibling('dd').text[:-3]
                
            #obtener memory clock
            memory_clock = soup.find('dt', text='Memory Clock')
            if memory_clock is not None:
                memory_clock = memory_clock.find_next_sibling('dd').text[:-3]
                
            # Obtener el performance level
            perf_level = soup.find('meta', {'itemprop': 'ratingValue'})
            if perf_level is not None:
                perf_level = perf_level.text
                perf_level_pattern = re.compile(r'(\d+\.\d+/\d+)')
                perf_level_match = perf_level_pattern.search(perf_level)

                if perf_level_match is not None:
                    perf_level = perf_level_match.group(1)
                    perf_level = perf_level[:-3]

                    
            gpu = {
            'nombre': nombre.get_text(strip=True),
            'serie': serie,
            'vram': vram,
            'potencia': potencia,
            'length':length,
            'slots': slots,
            'eight_pin': eight_pin,
            'six_pin' : six_pin,
            'boost_clock': boost_clock,
            'memory_clock': memory_clock,
            'producer': producer,
            'perf_level': perf_level,
            'precio': price
            }
            gpus.append(gpu)

# Escribir los datos en un archivo CSV
with open('gpus.csv', mode='w', newline='', encoding='utf-8') as outfile:
    writer = csv.writer(outfile)
    # Escribir la cabecera del archivo CSV
    writer.writerow(['Name', 'Serie', 'VRAM', 'TDP (W)', 'Length (mm)', 'slots', '8 Pin', '6 Pin', 'Boost Clock','Memory Clock', 'Producer', 'Performance Level /10','Price'])
    # Escribir cada fila en el archivo CSV
    for gpu in gpus:
        writer.writerow([gpu['nombre'], gpu['serie'], gpu['vram'], 
                         gpu['potencia'], gpu['length'], gpu['slots'], gpu['eight_pin'], gpu['six_pin'],
                         gpu['boost_clock'], gpu['memory_clock'], gpu['producer'], gpu['perf_level'], gpu['precio']])


In [13]:
#exporta los datos necesarios de las motherboards 

import csv
import requests
from bs4 import BeautifulSoup

def make_soup(url):
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup

def scrape_motherboards():
    url = "https://www.pc-kombo.com/us/components/motherboards"
    soup = make_soup(url)

    motherboards = []
    for i in soup.find("ol", {'id': 'hardware'}).find_all('li'):
        if(i != 'None' and i != -1):
            nombre = i.find("h5", {'class': 'name'})
            size = i.find("span", {'class': 'size'}).text
            socket = i.find("span", {'class': 'socket'}).text
            chipset = i.find("span", {'class': 'chipset'}).text
            ramslots = i.find("span", {'class': 'ramslots'}).text
            # buscar el precio en el HTML
            precio_html = i.find("span", {'class': 'price'})
            if precio_html is not None:
                # obtener el valor del precio y asignarlo a la variable precio
                precio = precio_html.text.strip()[4:]
                
            if precio != '':
                url_motherboard = (i.find('a', href=True)['href']).split(" ")[0]
                response = requests.get(url_motherboard)

                soup = BeautifulSoup(response.text, 'html.parser')

                """ # Obtener el form factor
                form_factor = soup.find('dt', text='Form Factor')
                if form_factor is not None:
                    form_factor = form_factor.find_next_sibling('dd').text"""

                # Obtener el tipo de memoria y su capacidad
                memory_type = soup.find('dt', text='Memory Type')
                if memory_type is not None:
                    memory_type = memory_type.find_next_sibling('dd').text
                    
                memory_capacity = soup.find('dt', text='Memory Capacity')
                if memory_capacity is not None:
                    memory_capacity = memory_capacity.find_next_sibling('dd').text  
                
                # Obtener el productor
                producer = soup.find("dd",{'itemprop':'brand'} ).text
                    
            

                motherboard = {
                    'nombre': nombre.get_text(strip=True),
                    'size' :size,
                    'socket': socket,
                    'chipset': chipset,
                    'ramslots': ramslots,
                    'memory_type': memory_type,
                    'memory_capacity':memory_capacity,
                    'producer': producer,
                    'precio': precio
                }
                
                motherboards.append(motherboard)

    # Escribir los datos en un archivo CSV
    with open('motherboards.csv', mode='w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        # Escribir la cabecera del archivo CSV
        writer.writerow(['Nombre', 'Size','Socket', 'Chipset', 'Ramslots','Memory_Type', 'Memory_Capacity','Producer', 'Precio'])
        # Escribir cada fila en el archivo CSV
        for motherboard in motherboards:
            writer.writerow([motherboard['nombre'],motherboard['size'],motherboard['socket'], motherboard['chipset'],motherboard['ramslots'],  
                             motherboard['memory_type'], motherboard['memory_capacity'],motherboard['producer'], motherboard['precio']])
            
scrape_motherboards()

In [25]:
#exporta los datos necesarios de las ram 

import csv
import requests
from bs4 import BeautifulSoup

def make_soup(url):
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup

def scrape_rams():
    url = "https://www.pc-kombo.com/us/components/rams"
    soup = make_soup(url)

    rams = []
    for i in soup.find("ol", {'id': 'hardware'}).find_all('li'):
        if(i != 'None' and i != -1):
            nombre = i.find("h5", {'class': 'name'})
            size = i.find("span", {'class': 'size'}).text[:-2]
            types = i.find("span", {'class': 'type'}).text
            
          
            # buscar el precio en el HTML
            precio_html = i.find("span", {'class': 'price'})
            if precio_html is not None:
                # obtener el valor del precio y asignarlo a la variable precio
                precio = precio_html.text.strip()[4:]
                
            if precio != '':
                url_ram = (i.find('a', href=True)['href']).split(" ")[0]
                response = requests.get(url_ram)

                soup = BeautifulSoup(response.text, 'html.parser')

                #obtener el clock
                clock = soup.find('dt', text='Clock')
                if clock is not None:
                    clock = clock.find_next_sibling('dd').text
                
                #obtener los sticks
                sticks = soup.find('dt', text='Sticks')
                if sticks is not None:
                    sticks = sticks.find_next_sibling('dd').text 
                
                # Obtener el productor
                producer = soup.find("dd",{'itemprop':'brand'} ).text
                    
            

                ram = {
                    'nombre': nombre.get_text(strip=True),
                    'size' :size,
                    'type' : types,
                    'clock': clock,
                    'sticks': sticks,
                    'producer': producer,
                    'precio': precio
                }
                
                rams.append(ram)


    # Escribir los datos en un archivo CSV
    with open('rams.csv', mode='w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        # Escribir la cabecera del archivo CSV
        writer.writerow(['Name', 'Size', 'Type', 'Clock', 'Sticks', 'Producer', 'Price'])
        # Escribir cada fila en el archivo CSV
        for ram in rams:
            writer.writerow([ram['nombre'],ram['size'],ram['type'],ram['clock'],ram['sticks']
                            ,ram['producer'],ram['precio']])
            
scrape_rams()

In [28]:
#exporta los datos necesarios de las HDD

import csv
import requests
from bs4 import BeautifulSoup

def make_soup(url):
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup

def scrape_hdds():
    url = "https://www.pc-kombo.com/us/components/hdds"
    soup = make_soup(url)

    hdds = []
    for i in soup.find("ol", {'id': 'hardware'}).find_all('li'):
        if(i != 'None' and i != -1):
            nombre = i.find("h5", {'class': 'name'})
            size = i.find("span", {'class': 'size'}).text[:-2]
            rpm = i.find("span", {'class': 'rpm'}).text
            
          
            # buscar el precio en el HTML
            precio_html = i.find("span", {'class': 'price'})
            if precio_html is not None:
                # obtener el valor del precio y asignarlo a la variable precio
                precio = precio_html.text.strip()[4:]
                
            if precio != '':
                url_hdd = (i.find('a', href=True)['href']).split(" ")[0]
                response = requests.get(url_hdd)
                soup = BeautifulSoup(response.text, 'html.parser')

                
                # Obtener el productor
                producer = soup.find("dd",{'itemprop':'brand'} ).text
                    
            

                hdd = {
                    'nombre': nombre.get_text(strip=True),
                    'size' :size,
                    'rpm' : rpm,
                    'producer': producer,
                    'precio': precio
                }
                
                hdds.append(hdd)


    # Escribir los datos en un archivo CSV
    with open('hdds.csv', mode='w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        # Escribir la cabecera del archivo CSV
        writer.writerow(['Name', 'Size', 'Rpm', 'Producer', 'Price'])
        # Escribir cada fila en el archivo CSV
        for hdd in hdds:
            writer.writerow([hdd['nombre'],hdd['size'],hdd['rpm']
                            ,hdd['producer'],hdd['precio']])
            
scrape_hdds()

In [29]:
#exporta los datos necesarios de las SSD

import csv
import requests
from bs4 import BeautifulSoup

def make_soup(url):
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup

def scrape_ssds():
    url = "https://www.pc-kombo.com/us/components/ssds"
    soup = make_soup(url)

    ssds = []
    for i in soup.find("ol", {'id': 'hardware'}).find_all('li'):
        if(i != 'None' and i != -1):
            nombre = i.find("h5", {'class': 'name'})
            size = i.find("span", {'class': 'size'}).text[:-2]
            
          
            # buscar el precio en el HTML
            precio_html = i.find("span", {'class': 'price'})
            if precio_html is not None:
                # obtener el valor del precio y asignarlo a la variable precio
                precio = precio_html.text.strip()[4:]
                
            if precio != '':
                url_hdd = (i.find('a', href=True)['href']).split(" ")[0]
                response = requests.get(url_hdd)
                soup = BeautifulSoup(response.text, 'html.parser')
                
                #obtener el form factor
                form_factor = soup.find('dt', text='Form Factor')
                if form_factor is not None:
                    form_factor = form_factor.find_next_sibling('dd').text
                
                #obtener el protocolo
                protocolo = soup.find('dt', text='Protocol')
                if protocolo is not None:
                    protocolo = protocolo.find_next_sibling('dd').text

                
                # Obtener el productor
                producer = soup.find("dd",{'itemprop':'brand'} ).text
                    
            

                ssd = {
                    'nombre': nombre.get_text(strip=True),
                    'size' :size,
                    'form_factor':form_factor,
                    'protocolo':protocolo,
                    'producer': producer,
                    'precio': precio
                }
                
                ssds.append(ssd)


    # Escribir los datos en un archivo CSV
    with open('ssds.csv', mode='w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        # Escribir la cabecera del archivo CSV
        writer.writerow(['Name', 'Size', 'Form Factor','Protocol', 'Producer', 'Price'])
        # Escribir cada fila en el archivo CSV
        for ssd in ssds:
            writer.writerow([ssd['nombre'],ssd['size'],ssd['form_factor'],ssd['protocolo']
                            ,ssd['producer'],ssd['precio']])
            
scrape_ssds()

In [35]:
#exporta los datos necesarios de las Power Supplies

import csv
import requests
from bs4 import BeautifulSoup

def make_soup(url):
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup

def scrape_ssds():
    url = "https://www.pc-kombo.com/us/components/psus"
    soup = make_soup(url)

    pss = []
    for i in soup.find("ol", {'id': 'hardware'}).find_all('li'):
        if(i != 'None' and i != -1):
            nombre = i.find("h5", {'class': 'name'})
            size = i.find("span", {'class': 'size'}).text[:-2]
            watts = i.find("span", {'class': 'watt'}).text[:-1]
            
          
            # buscar el precio en el HTML
            precio_html = i.find("span", {'class': 'price'})
            if precio_html is not None:
                # obtener el valor del precio y asignarlo a la variable precio
                precio = precio_html.text.strip()[4:]
                
            if precio != '':
                url_hdd = (i.find('a', href=True)['href']).split(" ")[0]
                response = requests.get(url_hdd)
                soup = BeautifulSoup(response.text, 'html.parser')
                
                #obtener el efficiency
                efficiency = soup.find('dt', text='Efficiency Rating')
                if efficiency is not None:
                    efficiency = efficiency.find_next_sibling('dd').text
                
                #obtener los 8 pin
                eight_pin = soup.find('dt', text='PCI-E cables 8-pin')
                if eight_pin is not None:
                    eight_pin = eight_pin.find_next_sibling('dd').text
                else:
                    eight_pin = 0
                
                #obtener los 6 pin
                six_pin = soup.find('dt', text='PCI-E cables 6-pin')
                if six_pin is not None:
                    six_pin = six_pin.find_next_sibling('dd').text
                else:
                    six_pin = 0

                
                # Obtener el productor
                producer = soup.find("dd",{'itemprop':'brand'} ).text
                    
            

                ps = {
                    'nombre': nombre.get_text(strip=True),
                    'size' :size,
                    'watts':watts,
                    'efficiency':efficiency,
                    'eight_pin': eight_pin,
                    'six_pin': six_pin,
                    'producer': producer,
                    'precio': precio
                }
                
                pss.append(ps)


    # Escribir los datos en un archivo CSV
    with open('powerSupplies.csv', mode='w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        # Escribir la cabecera del archivo CSV
        writer.writerow(['Name', 'Size', 'Watts','Efficiency Rating','8 Pin', '6 Pin', 'Producer', 'Price'])
        # Escribir cada fila en el archivo CSV
        for ps in pss:
            writer.writerow([ps['nombre'],ps['size'],ps['watts'],ps['efficiency'],
                            ps['eight_pin'],ps['six_pin'],ps['producer'],ps['precio']])
            
scrape_ssds()

In [46]:
#exporta los datos necesarios de los coolers

import csv
import requests
from bs4 import BeautifulSoup

def make_soup(url):
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup

def scrape_coolers():
    url = "https://www.pc-kombo.com/us/components/cpucoolers"
    soup = make_soup(url)

    coolers = []
    for i in soup.find("ol", {'id': 'hardware'}).find_all('li'):
        if(i != 'None' and i != -1):
            nombre = i.find("h5", {'class': 'name'})
      
            
          
            # buscar el precio en el HTML
            precio_html = i.find("span", {'class': 'price'})
            if precio_html is not None:
                # obtener el valor del precio y asignarlo a la variable precio
                precio = precio_html.text.strip()[4:]
                
            if precio != '':
                url_hdd = (i.find('a', href=True)['href']).split(" ")[0]
                response = requests.get(url_hdd)
                soup = BeautifulSoup(response.text, 'html.parser')
                
                #obtener los sockets
                sockets = soup.find('dt', text='Supported Sockets')
                if sockets is not None:
                    sockets = sockets.find_next_sibling('dd').text[11:]
                
                #obtener el height
                height = soup.find('dt', text='Height')
                if height is not None:
                    height = height.find_next_sibling('dd').text[:-2]
                
                #obtener el tdp
                tdp = soup.find('dt', text='TDP')
                if tdp is not None:
                    tdp = tdp.find_next_sibling('dd').text[:-2]

                
                # Obtener el productor
                producer = soup.find("dd",{'itemprop':'brand'} ).text
                    
            

                cooler = {
                    'nombre': nombre.get_text(strip=True),
                    'sockets' :sockets,
                    'height':height,
                    'tdp':tdp,
                    'producer': producer,
                    'precio': precio
                }
                
                coolers.append(cooler)


    # Escribir los datos en un archivo CSV
    with open('coolers.csv', mode='w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        # Escribir la cabecera del archivo CSV
        writer.writerow(['Name', 'Sockets', 'Height','TDP','Producer', 'Price'])
        # Escribir cada fila en el archivo CSV
        for cooler in coolers:
            writer.writerow([cooler['nombre'],cooler['sockets'],cooler['height'],cooler['tdp'],
                         cooler['producer'],cooler['precio']])
            
scrape_coolers()

In [5]:
#exporta los datos necesarios de las cases

import csv
import requests
from bs4 import BeautifulSoup

def make_soup(url):
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup

def scrape_cases():
    url = "https://www.pc-kombo.com/us/components/cases"
    soup = make_soup(url)

    cases = []
    for i in soup.find("ol", {'id': 'hardware'}).find_all('li'):
        if(i != 'None' and i != -1):
            nombre = i.find("h5", {'class': 'name'})
      
          
            # buscar el precio en el HTML
            precio_html = i.find("span", {'class': 'price'})
            if precio_html is not None:
                # obtener el valor del precio y asignarlo a la variable precio
                precio = precio_html.text.strip()[4:]
                
            if precio != '':
                url_hdd = (i.find('a', href=True)['href']).split(" ")[0]
                response = requests.get(url_hdd)
                soup = BeautifulSoup(response.text, 'html.parser')
                
                #obtener el width
                width = soup.find('dt', text='Width')
                if width is not None:
                    width = width.find_next_sibling('dd').text[:-2]
                    
                #obtener el Depth
                depth = soup.find('dt', text='Depth')
                if depth is not None:
                    depth = depth.find_next_sibling('dd').text[:-2]
                
                #obtener el height
                height = soup.find('dt', text='Height')
                if height is not None:
                    height = height.find_next_sibling('dd').text[:-2]
                
                #obtener la máxima length de gpu prmitida
                max_gpu_length = soup.find('dt', text='Supported GPU length')
                if max_gpu_length is not None:
                    max_gpu_length = max_gpu_length.find_next_sibling('dd').text[:-2]

                
                # Obtener el productor
                producer = soup.find("dd",{'itemprop':'brand'} ).text
                    
            

                case = {
                    'nombre': nombre.get_text(strip=True),
                    'width' :width,
                    'depth':height,
                    'height':height,
                    'max_gpu_length': max_gpu_length,
                    'producer': producer,
                    'precio': precio
                }
                
                cases.append(case)


    # Escribir los datos en un archivo CSV
    with open('cases.csv', mode='w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        # Escribir la cabecera del archivo CSV
        writer.writerow(['Name', 'Width mm', 'Depth mm','Height mm',' Maximum GPU Length','Producer', 'Price'])
        # Escribir cada fila en el archivo CSV
        for case in cases:
            writer.writerow([case['nombre'],case['width'],case['depth'],case['height'],
                         case['max_gpu_length'],case['producer'],case['precio']])
            
scrape_cases()

In [ ]:
"""url = "https://www.pc-kombo.com/us/product/motherboard/4719072733667_MSI%20B550-A%20Pro"
soup = make_soup(url)

price = soup.find("body").find("section", {'class': 'column col-12 col-md-12'}).find('span', {'itemprop':'price'}).text
print("PRECIO:", price)
list_a = soup.find("body").find('div', {'class': 'container grid-xl'}).find('div', {'class': 'columns'}).find_all('dd')
list_b = soup.find("body").find('div', {'class': 'container grid-xl'}).find('div', {'class': 'columns'}).find_all('dt')

for i in range(len(list_a)):
    print(list_b[i].text, " ", list_a[i].text)
    """

In [ ]:
"""def scrapping_all_components(code):

    url = "https://www.pc-kombo.com/us/components/" + code
    soup = make_soup(url)
    columns_scrapping = ['Producer', 'Length', 'Slots', '8-pin connectors', '6-pin connectors', 'Vram',
                         'Boost Clock', 'Memory Clock']
    lista = [m for m in soup.find("body").find("ol", {'id': 'hardware'}).find_all('li')]
    final = []
    ibi = 0
    
    for i in lista:

        i = (i.find('a', href=True)['href']).split(" ")[0]

        if(i != 'None' and i != -1):
            ibi+=1
            print(i)

            url_2 = i
            soup_2 = make_soup(url_2)
            
            name = soup_2.find("body").find("h1", {"itemprop":"name"}).text
            
            rating = str(soup_2.find("body").find('p', {'itemprop' : 'aggregateRating'}).text).replace(" ", "")
            rating = rating.replace('\n', '').replace('\r', '')
            rating = rating[-2:-9:-1]
            price = soup_2.find("body").find("section", {'class': 'column col-12 col-md-12'}).find('span', {'itemprop':'price'})
            list_a = soup_2.find("body").find('div', {'class': 'container grid-xl'}).find('div', {'class': 'columns'}).find_all('dd')
            list_b = soup_2.find("body").find('div', {'class': 'container grid-xl'}).find('div', {'class': 'columns'}).find_all('dt')
            
            #ELEMNTOS PARA LA TABLA: NAME, PRODUCER, MODEL, LENGTH, SLOTS, 8_PIN, 6_PIN,
            # VRAM, TDP, BOOST_CLOCK, MEMORY_CLOCK, PERFORMANCE_LEVEL, PRICE
            if price == None:
                print(name + "," + str(price) + "," + rating[::-1] + ",",  end = "")
            else:
                print(name + "," + str(price.txt) + "," + rating[::-1] + ",", end = "")

            for i in range(len(list_a)):
                
                if list_b[i].text in columns_scrapping:
                    print(list_a[i].text, end = "")
                    print(",", end="")
                   
            print("")

    print(ibi)
    
scrapping_all_components("gpus")"""

In [ ]:
#ELEMNTOS PARA LA TABLA: NAME, PRODUCER, MODEL, LENGTH, SLOTS, 8_PIN, 6_PIN, VRAM, TDP, BOOST_CLOCK, MEMORY_CLOCK, PERFORMANCE_LEVEL, PRICE

# ORDENADOR PRINCIPAL

In [ ]:
import platform
import cpuinfo
import psutil
import GPUtil

def unidades(bytes, suffix="B"):
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor

print(f"NOMBRE DEL EQUIPO: {platform.node()}")
print(f"SISTEMA OPERATIVO: {platform.system()} con VERSIÓN {platform.version()}")
print(f"PROCESADOR: {cpuinfo.get_cpu_info()['brand_raw']} CON {psutil.cpu_count(logical=False)} NÚCLEOS")
print(f"GRAFICA: {GPUtil.getGPUs()[0].name}")
print(f"MEMORIA RAM: {unidades(psutil.virtual_memory().total)}")
partition_usage = psutil.disk_usage(psutil.disk_partitions()[0].mountpoint)
print(f"INFORMACION DEL DISCO: ")
print(f"  TAMAÑO TOTAL: {unidades(partition_usage.total)}")
print(f"  USADO: {unidades(partition_usage.used)}")
print(f"  DISPONIBLE: {unidades(partition_usage.free)}")
